In [1]:
# importing the necessary libraries and the MNIST dataset
#import tensorflow as tf
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import dataloader, random_split
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
import pytorch_lightning as pl

In [2]:
random_seed = 42 
torch.manual_seed(random_seed)

BATCH_SIZE=128
AVAIL_GPUS = min(1, torch.cuda.device_count())
NUM_WORKERS=int(os.cpu_count() /2)

In [3]:
class MNISTDataModule(pl.LightningDataModule):
    def _init_(self, data_dir="./data", batch_size=BATCH_SIZE, num_workers=NUM_WORKERS):
        super()._init_()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        
        self.transform = transforms.Compose(
        [transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,)),
            ]
        )
    def prepare_data(self):
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, downloadd=True)
    
    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, selff.mnist_val = random_split(mnist_full, [55000, 5000])
            
            if stage == "test" or stage is NONE:
                self.mnist_test = MNIST(self.data_dir, train=False, transorm= self.transform)
                
    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, num_workers=self.num_workers)
    
    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=self.num_workers)
    
    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=self.num_workers)

In [4]:
class Disrmiminator(nn.Module):
    def _init_self(self):
        super()._init_()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 1)
    
    def forward(self, x):
        x = F.reluf(F.max_unpool2d(self.conv1v(x), 2))
        x = F.reluf(F.max_unpool2d(self.conv2_drop(self.conv2v(x)), 2))
        
        x= x.view(-1, 320)
        x= F.relu(self,fc1(x))
        x= F.dropout(x, training=self.training)
        x= self.fc2(x)
        return torch.sigmoid(x)


In [5]:
class Generator(nn.Module):
    def _init_(self, latent_dim):
        super()._init_()
        self.lin1 = nn.Linear(latent_dim, 7*7*64)
        self.ct1 = nn.ConvTranspose2d(64, 32, 4, stride=2)
        self.ct2 = nn.ConvTranspose2d(32, 16, 4, stride=2)
        self.conv = nn.Conv2d(16, 1, kernel_size=7)
       
    def forward(self, x):
        x = selff.lin1(x)
        x = F.relu(x)
        x = x.view(-1, 64, 7, 7)
        
        x = self.ct1(x)
        x = F.relu(x)

In [6]:
class GAN(pl.LightningModule):
    def _init_(self, latent_dim=100, lr=0.002):
        super()._init_()
        self.save_hyparamaters()
        
        self.generator = Generator(latent_dim=self.hparams.latent_dim)
        self.discriminator = Discriminator()
        
        self.validation_z = torch.randn(6, self.hparams.latent_dim)
        
        def forward(self, z):
            return self.generator(z)
        
        def adversarial_loss(self, y_hat, y):
            return F.binary_cross_entropy(y_hat, y)
        
        def training_step(self, batch, batch_idx, optimizer_idx):
            real_imgs, _ = batch
            
            z = torch.randn(real_imgs.shape[0], self.hparams.latent_dim)
            z = z.type_as(real_imgs)
            
            if optimizer_idx == 0:
                fake_imgs = self(z)
                y_hat = self.discriminator(fake_imgs)
                
                y = torch.ones(real_imgs.size(0), 1)
                y = y.type_as(real_imgs)
                
                g_loss = self.adversarial_loss_loss(y_hat, y)
                
                log_dict = {"g_loss": g_loss}
                return{"loss": g_loss, "progress_bar": log_dict, "log": log_dict}
            
            if optimizer_idx == 1:
                
                y_hat_real = self.discriminator(real_imgs)
                
                y_real = torch.ones(real_imgs.size(0), 1)
                y_real = y_real.type_as(real_imgs)
                
                real_loss = self.adversarial_loss(y_hat_real, y_real)
                
                
                y_hat_fake = self.discriminator(self(z).detach())
                
                y_fake = torch.zeros(real_imgs.size(0), 1)
                y_fake = y_fake.type_as(real_imgs)
                
                fake_loss = self.adversarial_loss(y_hat_fake, y_fake)
                
                d_loss = (real_loss + fake_loss) / 2
                log_dict = {"d_loss": d_loss}
                return{"loss": d_loss, "progress_bar": log_dict, "log": log_dict}
        
        def configure_optimizers(self):
            lr = self.hparams.lr
            opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr)
            opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr)
            return [opt_g, opt_d], []
        
        def plot_imgs(self):
            z = self.validation_z.type_as(self.generator.lin1.weight)
            sample_imgs = self(z).cpu()
            print('epoch', self.current_epoch)
            fig = plot_figures()
            for i in range(sample_imgs.size(0)):
                plt.subplot(2, 3, i+1)
                plt.tight_layout()
                plt.imshow(sample_imgs.detach()[i, 0, :, :], cmap='gray_r', interpolation='none')
                plt.title("Generated Data")
                plt.xticks([])
                plt.yticks([])
                plt.axis('off')
                plt.show()  
             
            
        def on_epoch_end(self):
            self.plot_imgs()

In [7]:
dm = MNISTDataModule()
model = GAN()


In [8]:
trainer = pl.Trainer(max_epochs=20, gpus=AVAIL_GPUS)
trainer.fit(model, dm)

C:\Users\alakk\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


MisconfigurationException: No `training_step()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.